### The example script below demonstrates how a randomized plate layout can be generated from an initial description (see input_file.csv) of the experiment 

In [1]:
import pandas as pd
from datarail.experimental_design import process_assay as pa
from datarail.experimental_design import hpdd_utils as hu

The input file (see **`input_file.csv`**) should be broad description of the planned experimental design and should contain the following columns<br>
>__agent__ : lists the names of drugs, including positve controls.<br>
**max_dose__um** : lists the highest dose for each agent.<br>
__num_doses__ : lists the number of doses for each agent.<br>
__role__ : lists the intended role for each agent
   `'treatment', or 'positive_control'`<br>
__num_replicates__ : lists number of times a drug's
   dosing scheme is replicated on the same plate.<br>
__exlcude_doses__ :  list doses (comma seperated) to be excluded for a given drug (**OPTIONAL**; defaults to removing lowest doses if num_doses < 9).<br>



In [2]:
# construct mapping between wells and treatment conditions
df_well = pa.construct_well_level_df('input_file.csv')
df_well = pa.add_negative_control(df_well, control_name='DMSO')

In [10]:
# define drug and dose to be used as harsh treatment for plate barcode (OPTIONAL)
fingerprint_drug = 'Staurosporine'
fingerprint_dose = 1 # in micromolar

In [11]:
# create randomized plate layout for 3 replicates
plate_names = ['RL_A', 'RL_B', 'RL_C']

In [8]:
dfr = pa.randomize_wells(df_well, plate_names=plate_names,
                         fingerprint_drug=fingerprint_drug, # optional
                         fingerprint_dose=fingerprint_dose) # optional

In [9]:
dfr.head()


,agent,concentration,identifier,plate,role,well
0,Staurosporine,1.0,Staurosporine_0,RL_A,fingerprint,A01
1,Staurosporine,1.0,Staurosporine_1,RL_A,fingerprint,A02
2,,0.0,0,RL_A,,A03
3,,0.0,0,RL_A,,A04
4,Staurosporine,1.0,Staurosporine_2,RL_A,fingerprint,A05


In [12]:
# load a file (see drug_stock.csv) with name of the agents and the corresponding stock concentrations
# The name of the agents in the file should match those in design dataframe
df_stock = pd.read_csv('drug_stock.csv')

In [ ]:
# If the D300 is going to be used, run the code below to generate the hpdd file
hu.export_hpdd(dfr, df_stock, 'RL_D300.hpdd')